# Métodos iterativos para sistemas lineales

Aunque la eliminación Gaussiana es un método que da la solución **exacta** de un sistema lineal $\mathbf{Ax} = \mathbf{b}$, es un método difícil de implementar y no es simple de comprender o deducir para todos.

Así, existen métodos **iterativos**, mucho más sencillos de entender e implementar, que generan una sucesión de vectores $\{ \mathbf{x}^{(k)} \}_{k=1}^{\infty}$ que, bajo ciertas condiciones, convergen a la solución deseada.

A diferencia de los métodos anteriores donde hemos construido sucesiones de números reales, tener ahora una sucesión **vectorial** puede parecer complicado pero es básicamente lo mismo.

## Preludio Computacional I: Arreglos de arreglos vs Arreglos n-dimensionales

Como vamos a trabjar con sucesiones de vectores, una pregunta relevante resulta ser cómo representar una sucesión de vectores en la computadora.

Hay dos opciones principales para representar una sucesión de vectores:

### Opción 1: arreglo de arreglos

Cuando aprendimos sobre los arreglos, comentamos que los arreglos solo podían contener objetos del mismo tipo y, en principio, solo objetos `Int`, `Float`, `String` o `Char`. Sin embargo, también es posible que un arreglo contenga **arreglos** o Listas o cualquier otro objeto iterable como elemento dentro de sí. 

Nuevamente, ya que un arreglo no puede tener elementos de tipo distinto, los elementos de un arreglo no pueden serarreglos y listas o arreglos y rangos: debe de tener objetos del mismo tipo.



In [16]:
# arreglo de arreglos
arr = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
display(arr)
println(typeof(arr))

4-element Array{Array{Int64,1},1}:
 [1, 2, 3]
 [4, 5, 6]
 [7, 8, 9]
 [10, 11, 12]

Array{Array{Int64,1},1}


Si accedemos al i-ésimo elemento del arreglo, obtendremos otro arreglo

In [2]:
display(arr[2])

3-element Array{Int64,1}:
 4
 5
 6

Podemos acceder ahora al j-ésimo elemento del i-ésimo elemento

In [3]:
# los primero corches indexan sobre `arr` y los segundos indexan sobre `arr[2]`
println(arr[2][3])

6


Podemos utilizar esto para construir arreglos paso a paso con la función `push!`

In [18]:
function arregloDeArreglosPush(n)
    # Inicializo el arreglo con un elemento
    res = [[1.0,1.0,2.0]]
    # construyo n-1 términos adicionales
    for i in 2:n
        # el nuevo término es el vector anterior más [-1,2,3]
        push!(res,res[i-1]+[-1.0,2.0,3.0])
    end
    return res
end

arregloDeArreglosPush (generic function with 1 method)

In [19]:
res = arregloDeArreglosPush(10)
display(res)

10-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 2.0]
 [0.0, 3.0, 5.0]
 [-1.0, 5.0, 8.0]
 [-2.0, 7.0, 11.0]
 [-3.0, 9.0, 14.0]
 [-4.0, 11.0, 17.0]
 [-5.0, 13.0, 20.0]
 [-6.0, 15.0, 23.0]
 [-7.0, 17.0, 26.0]
 [-8.0, 19.0, 29.0]

### Opción 2: usar un arreglo 2D (una matriz)

Si todos los elementos de la sucesión vectorial son del mismo tamaño (que generalmente así es), entonces los puedo almacenar en un arreglo 2D

In [20]:
# notación para matrices. Columnas se separan por espacios y renglones por ';'
nuevoarr = [[1 2 3] ; [4 5 6] ; [7 8 9] ; [10 11 12]]

4×3 Array{Int64,2}:
  1   2   3
  4   5   6
  7   8   9
 10  11  12

Ahora, cada renglón representa un vector de la sucesión

In [21]:
# accedo al tercer renglón
display(nuevoarr[3,:])

3-element Array{Int64,1}:
 7
 8
 9

Puedo añadirle un nuevo renglón representado por un vector mediante la función `vcat`, aunque debo transponer el vector antes de añadirlo usando la función `transpose`

In [22]:
vcat(nuevoarr,transpose([13,14,15]))

5×3 Array{Int64,2}:
  1   2   3
  4   5   6
  7   8   9
 10  11  12
 13  14  15

Notemos que esto no cambia a la variable `nuevoarr`

In [23]:
display(nuevoarr)

4×3 Array{Int64,2}:
  1   2   3
  4   5   6
  7   8   9
 10  11  12

Esto es diferente a la mecánica de la función `push!`, la cuál si cambia el arreglo al que añadimos de manera automática.

Para añadirle el vector `[13,14,15]` de manera permanente a `nuevoarr`, debo redefinir la variable:

In [24]:
nuevoarr = vcat(nuevoarr,transpose([13,14,15]))
display(nuevoarr)

5×3 Array{Int64,2}:
  1   2   3
  4   5   6
  7   8   9
 10  11  12
 13  14  15

Podemos usar lo anterior para construir una matriz paso a paso

In [25]:
function matrizVcat(n)
    # Inicializo la matriz conteniendo solo un vector
    res = [1.0 1.0 2.0]
    # construyo n términos
    for i in 2:n
        # el nuevo término es el vector anterior más [-1,2,3]
        res = vcat(res,transpose(res[i-1,:]+[-1.0,2.0,3.0]))
    end
    return res
end

matrizVcat (generic function with 1 method)

In [26]:
res = matrizVcat(10)
display(res)

10×3 Array{Float64,2}:
  1.0   1.0   2.0
  0.0   3.0   5.0
 -1.0   5.0   8.0
 -2.0   7.0  11.0
 -3.0   9.0  14.0
 -4.0  11.0  17.0
 -5.0  13.0  20.0
 -6.0  15.0  23.0
 -7.0  17.0  26.0
 -8.0  19.0  29.0

## Preludio Computacional II: la librería `LinearAlgebra` de Julia

Cuando trabajamos con vectores, hay operaciones que muchas veces resultan necesarias como calcular su norma euclideana o realizar un producto punto.

Julia tiene, dentro de sí misma, una librería llamada `LinearAlgebra` con código útil para realizar operaciones de Álgebra Lineal. Su [Documentación](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/) indica como usarla a profundidad, pero aquí mostraremos un par de ejemplos sencillos


In [27]:
# importamos la libreria. No hace falta instalarla pues ya está incluída con Julia
using LinearAlgebra

In [28]:
vec1 = [1.0,1.0,0.0]
vec2 = [1.0,0.0,1.0]
# `norm` nos regresa la norma euclideana de un vector
println(norm(vec1))
println(norm(vec2))
# `dot` nos permite calcular el producto punto de dos vectores
println(dot(vec1,vec2))
# `norm(vec1-vec2)` nos regresa la distancia euclideana entre vec1 y vec2
println(norm(vec1-vec2))

1.4142135623730951
1.4142135623730951
1.0
1.4142135623730951


Por el momento, solo utilizaremos estas dos funciones pero en el **futuro (siguiente clase)** esta librería será importante.

## Método de Jacobi

Supongamos que tenemos un sistema **con solución única** $\mathbf{Ax} = \mathbf{b}$. Explícitamente, sabemos que el sistema se muestra de la siguiente forma

$$
\begin{align}
a_{11} x_1 + a_{12} x_2 + \ldots + a_{1n} x_n &= b_1 \\
a_{21} x_1 + a_{22} x_2 + \ldots + a_{2n} x_n &= b_2 \\
\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \\
a_{n1} x_1 + a_{n2} x_2 + \ldots + a_{nn} x_n &= b_n \\
\end{align}
$$

Por simplicidad, supongamos también que todos los elementos de la diagonal son distintos de cero ($a_{ii} \neq 0$). Para un sistema con solución única siempre podemos cambiar el orden de las ecuaciones (intercambiar renglones de $\mathbf{A}$) para que esto se cumpla. 

Así, podemos "despejar" cualquier $x_i$ de la i-ésima ecuación, poniéndolo en términos de los otros $x_{j}$ con $i \neq j$ de la siguiente forma:

$$
\begin{align}
 x_1 &= \frac{1}{a_{11}} \left(b_1 - ( a_{12} x_2 + a_{13} x_3 + \ldots + a_{1n} x_n) \right)  \\
 x_2 &= \frac{1}{a_{22}} \left(b_2 - ( a_{21} x_1 + a_{23} x_3  + \ldots + a_{2n} x_n) \right)  \\
 \vdots &\quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \\
 x_n &= \frac{1}{a_{nn}} \left(b_n - ( a_{n1} x_1 + a_{n2} x_2  + \ldots + a_{n n-1} x_{n-1}) \right)  \\
\end{align}
$$

Podemos generalizar el despeje anterior para cualquier $x_i$ mediante la siguiente fórmula

$$
x_{i} = \frac{1}{a_{ii}}\left( b_i - \sum_{j=1 \\ j\neq i}^{n} a_{ij}x_j \right)
$$

Así, utilizaremos esta expresión para generar una **sucesión recursiva de vectores** $\{\mathbf{x}^{(t)}\}_{t=1}^{\infty}$ Definida, entrada a entrada, mediante:

$$
x_i^{(t)} = \begin{cases}
x^{(\text{inicial})}_i & \text{si } t=1 \\
& \\
\frac{1}{a_{ii}}\left( b_i - \sum_{j=1 \\ j\neq i}^{n} a_{ij}x^{(t-1)}_j \right) & \text{si } t>1 \\
\end{cases}
$$

Antes de implementar esta sucesión directamente, la reduciremos a una forma más compacta.

### Ejercicio 1

Escribe las ecuaciones 

$$
\begin{align}
    x_{1} &= \frac{1}{a_{11}}\left( b_1 - \sum_{j=1 \\ j\neq 1}^{n} a_{1j} x_j \right) \\
    x_{2} &= \frac{1}{a_{22}}\left( b_2 - \sum_{j=1 \\ j\neq 2}^{n} a_{2j} x_j \right) \\
    &\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \\
 x_{n} &= \frac{1}{a_{nn}}\left( b_n - \sum_{j=1 \\ j\neq n}^{n} a_{nj} x_j \right) \\
\end{align}
$$

De una forma matricial:

$$
\mathbf{x} = \mathbf{Cx} +  \mathbf{d} 
$$

Con $\mathbf{C}$ una matriz de $n \times n$ y $\mathbf{d}$ un vector de $n$ entradas. Debes mostrar explícitamente los elementos de $\mathbf{C}$ y de $\mathbf{d}$

**Sugerencia:** La matriz $C$ debe de tener $0$s en la diagonal

Después de encontrar la matriz $\mathbf{C}$, la sucesión recursiva de vectores puede escribirse de manera compacta como:

$$
\mathbf{x}^{(t)} = \begin{cases}
\mathbf{x}^{(\text{inicial})} & \text{si } t=1 \\
& \\
\mathbf{C}\mathbf{x}^{(t-1)} +  \mathbf{d} & \text{si } t>1 \\
\end{cases}
$$


Dicha sucesión puede converger a la solución del sistema de ecuaciones bajo ciertas condiciones. A este método para paroximar una solución del sistema lineal se le conoce como **Método de Jacobi** o **iteraciones de Jacobi**

### Ejercicio 2

Implementa una función `jacobi(A,b,T,x_inicial)` donde $A$ es una matriz de $n \times n$, $b$ es un vector de longitud $n$, $T$ es un número natural y $x_{\text{inicial}}$ un vector de $n$ entradas. La función debe de generar los primeros $T$ términos de la sucesión vectorial $\mathbf{x}^{(t)}$ correspondiente al método de Jacobi.

La función debe de regresarte o bien un arreglo 1D `res` tal que `res[t]` es un vector de longitud $n$ correspondiente a $\mathbf{x}^{(t)}$ (un arreglo de arreglos) o bien un arreglo 2D `res` de $ T \times n $, en el que `res[t,:]` corresponde a $\mathbf{x}^{(t)}$ (una matriz)

**Sugerencia:** primero construye la matriz $\mathbf{C}$ y el vector $\mathbf{d}$ y utiliza la multiplicación de matrices por vectores y la suma de vectores para hacer tu código más compacto. Recuerda utilizar la función `copy` para hacer una copia de un arreglo a la hora de definir tanto $\mathbf{C}$ como $\mathbf{d}$

### Ejercicio 3

Implementa una función `jacobiTol(A,b,epsilon,x_inicial)` donde $A$ es una matriz de $n \times n$, $b$ es un vector de longitud $n$, $epsilon$ es un número real pequeño y $x_{\text{inicial}}$ un vector de $n$ entradas. La función debe de generar los términos de la sucesión vectorial $\mathbf{x}^{(t)}$ correspondiente al método de Jacobi hasta el término $m$ que cumpla que 

$$
\Vert\mathbf{A} \mathbf{x}^{(m)} - \mathbf{b} \Vert < epsilon
$$

La función debe de regresarte o bien un arreglo 1D `res` tal que `res[t]` es un vector de longitud $n$ correspondiente a $\mathbf{x}^{(t)}$ o bien un arreglo 2D `res` de $ m \times n $, en el que `res[t,:]` corresponde a $\mathbf{x}^{(t)}$


### Ejercicio 4

Utiliza cualquiera de las dos funciones anteriores para resolver el siguiente sistema

$$
\begin{align}
4x - y + z &= 7 \\
4x - 8y + z &= -21 \\
-2x +y + 5z &= 15
\end{align}
$$

Con solución exacta $x=2$, $y=4$, $z=3$. ¿La sucesión resultante converge? 

Grafica el error absoluto ($\Vert \mathbf{x}^{(t)} - \mathbf{x}_{\text{exacta}} \Vert$) como función del paso $t$ de la iteración y di algo sobre su tendencia.

## Método de Gauss-Siedel

Aunque parece útil y sencillo, el método de Jacobi puede resultar **inestable**. 

### Ejercicio 5

Reordena el sistema el ejercicio 4 de la siguiente forma:

$$
\begin{align}
-2x +y + 5z &= 15 \\
4x - y + z &= 7 \\
4x - 8y + z &= -21
\end{align}
$$

E intenta resolverlo usando la función `jacobi(A,b,T,x_inicial)` para un numero pequeño de iteraciones (alrededor de $10^2$). ¿El método converge?

Existe una manera de mejorarlo mucho haciendo un cambio teórico muy sencillo. Recordando la sucesión del método Jacobi entrada a entrada:

$$
x_i^{(t)} = \begin{cases}
x^{(\text{inicial})}_i & \text{si } t=1 \\
& \\
\frac{1}{a_{ii}}\left( b_i - \sum_{j=1 \\ j\neq i}^{n} a_{ij}x^{(t-1)}_j \right) & \text{si } t>1 \\
\end{cases}
$$

Notemos que para calcular $x_i^{(t)}$, utilizamos los $x_j^{(t-1)}$ para todos los $j\neq i $. Esto nos permite escribir el paso de la sucesión en términos de la matriz $\mathbf{C}$ y del vector $\mathbf{d}$ y hacer todo mucho más compacto. Además, nos permite actualizar todos los $x_i^{(t)}$ **simultáneamente**, es decir, con una misma operación para todos.

Sin embargo, qué sucede si en lugar de actualizar todo el vector $\mathbf{x}^{(t)}$ simultáneamente, vamos actualizando cada entrada $x_i^{(t)}$ en el orden que tienen en su índice. En ese caso, primero actualizaríamos

$$
x_1^{(t)} = \frac{1}{a_{11}}\left( b_1 - \sum_{j=1 \\ j\neq 1}^{n} a_{1j}x^{(t-1)}_j \right)
$$

Después

$$
x_2^{(t)} = \frac{1}{a_{22}}\left( b_2 - \sum_{j=1 \\ j\neq 1}^{n} a_{2j}x^{(t-1)}_j \right)
$$


En tercer lugar

$$
x_3^{(t)} = \frac{1}{a_{33}}\left( b_3 - \sum_{j=1 \\ j\neq 1}^{n} a_{3j}x^{(t-1)}_j \right)
$$

Y así hasta llegar al $n$. 

Sin embargo, en la fórmula para $x_{2}^{(t)}$, sabemos que aparece $x_{1}^{(t-1)}$. Si realizamos las operaciones una por una, a la hora de calcular $x_{2}^{(t)}$ ya habríamos calculado $x_{1}^{(t)}$, por lo que **podríamos utilizar ese valor** en lugar de $x_{1}^{(t-1)}$ en la fórmula de $x_{2}^{(t)}$. Así, tendríamos que

$$
x_2^{(t)} = \frac{1}{a_{22}}\left( b_2 - a_{21} x_{1}^{(t)} - \sum_{j=2 \\ j\neq 2}^{n} a_{2j}x^{(t-1)}_j \right)
$$

Para actualizar $x_{3}^{(t)}$, podemos repetir el mismo argumento y reemplazar ahora tanto $x_{1}^{(t-1)}$ como $x_{2}^{(t-1)}$ por $x_{1}^{(t)}$ y $x_{2}^{(t)}$ y obtener:

$$
x_3^{(t)} = \frac{1}{a_{33}}\left( b_3 - a_{31} x_{1}^{(t)} - a_{32} x_{2}^{(t)}  - \sum_{j=3 \\ j\neq 3}^{n} a_{3j}x^{(t-1)}_j \right)
$$
 
Este proceso se puede generalizar para la actualización de $x_i{(t)}$  mediante la siguiente fórmula

$$
x_{i}^{(t)}  = \frac{1}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x^{(t)}_j  - \sum_{j=i+1}^{n} a_{ij} x^{(t-1)}_j \right)
$$

Utilizando la ecuación anterior, la sucesión recursiva término a término se reescribe:

$$
x_i^{(t)} = \begin{cases}
x^{(\text{inicial})}_i & \text{si } t=1 \\
& \\
\frac{1}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x^{(t)}_j  - \sum_{j=i+1}^{n} a_{ij} x^{(t-1)}_j \right) & \text{si } t>1 \\
\end{cases}
$$

A este método para aproximar una solución del sistema lineal se le conoce como **Método de Gauss-Siedel**. En general, es mucho más estable que el método de Jacobi. 

### Ejercicio 6 

Implementa una función `gaussSiedel(A,b,T,x_inicial)` donde $A$ es una matriz de $n \times n$, $b$ es un vector de longitud $n$, $T$ es un número natural y $x_{\text{inicial}}$ un vector de $n$ entradas. La función debe de generar los primeros $T$ términos de la sucesión vectorial $\mathbf{x}^{(t)}$ correspondiente al método de Gauss-Siedel

La función debe de regresarte o bien un arreglo 1D `res` tal que `res[t]` es un vector de longitud $n$ correspondiente a $\mathbf{x}^{(t)}$ o bien un arreglo 2D `res` de $ T \times n $, en el que `res[t,:]` corresponde a $\mathbf{x}^{(t)}$

**Sugerencia:** tendrás que usar un ciclo `for` sobre las entradas de los vectores dentro de otro ciclo `for` sobre el número total de iteraciones para ir construyendo, entrada por entrada, un vector correspondiente a las actualizaciones. 

### Ejercicio 7 

Resuelve el sistema del ejercicio 4 utilizando la función `gaussSiedel`. Grafica el error absoluto. ¿El algoritmo converge más rápido que el método de Jacobi? ¿Qué sucede con el sistema del ejercicio 5? ¿El método de Gauss-Siedel si lo puede resolver?

## Operador `\` de Julia

Debido a que resolver sistemas lineales en general es muy importante en el cómputo científico, Julia tiene un operador integrado que resuelve sistemas lineales de manera muy rápida usando métodos más sofisiticados de los que les enseñé

In [15]:
# resolviendo el sistema del ejercicio 4
A1 = [[4.0 -1.0 1.0] ; [4.0 -8.0 1.0] ; [-2.0 1.0 5.0] ]
b1 = [7.0,-21.0,15.0]
# Asigna a `resul` la solución del sistema A1 * x = b1
resul = A1 \ b1
display(resul)

3-element Array{Float64,1}:
 2.0
 4.0
 3.0

### Ejercicio 8

Resuelve los sistemas del ejercicio 5 de esta clase y del ejercicio 7 de la clase 11 utilizando el operador `\` de Julia. ¿Las soluciones se obtienen más rápido (i.e. el tiempo de cómputo es menor) que con los otros métodos empleados?

## La visión general

Los sistemas lineales surgen en muchísimas áreas de las matemáticas y la física, por lo que es necesario conocer varios métodos para resolverlos.

De los métodos vistos aquí, ninguno es utilizado ampliamente en el día a día pues son muy lentos para resolver sistemas de tamaño grande ($n \geq 1000$). El de Gauss-Siedel es el que más se utiliza.

Hacia el final del curso, a la hora de ver Ecuaciones Diferenciales Parciales, nos empezarán a surgir muchos problemas cuya solución se reduce a resolver un conjunto de ecuaciones lineales.